# Capítulo 1 - O Problema

Temos no CEFET MG um grupo chamado Grupo de Computação Competitiva (GCC) que tem como uma de suas atividades a participação na maratona de programação.

A maratona de programação é uma competição onde equipes competem entre si em um período de tempo pré determinado a tentar resolver o maior número de problema em menos tempo.

O objetivo deste trabalho, é desenvolver um algoritmo de recomendação que ajude no treinamento dos competidores para a preparação para a maratona de programação.

Atualmente, o treinamento funciona da seguinte maneira. Existem alguns sites, chamados de juízes online, que funcionam como um repositório de problemas. O competidor então consegue ler o problema, desenvolver uma solução e submete-la para o juiz. O juiz então dará a resposta se a solução foi aceita ou não.

O problema do treinamento atual é que existem milhares de problemas a serem resolvidos. Isso faz com que o competidor perca tempo em problemas que não acrescentem em conteúdo para o seu desenvolvimento, ou então, escolha problemas muito difíceis para serem resolvidos, o que torna seu treinamento desmotivador.


# Capítulo 2 - Dataset


Foram coletados do site URI Online Judge, mais de 200 mil soluções de mais de 600 usuários. Cada solução é composta por:

 - **user_id**: usuário que resolveu o problema
 - **problem_id**: problema que foi resolvido
 - **date**: data de quando o problema foi resolvido
 - **category_id**: categoria do problema resolvido
 - **level**: dificuldade do problema resolvido
 - **solved**: quantidade total de usuários do juiz que resolveram o problema
 - **name**: nome do problema
 - **topics**: assuntos relacionados ao conteúdo do problema

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

def datefunc(date):
    d = datetime.strptime(date,'%m/%d/%y, %I:%M:%S %p')
    return d.strftime('%y/%m/%d %I:%M')

solutions_df = pd.read_csv('solutions.csv', delimiter='\t', converters={'date': datefunc})
solutions_df[:5]

,user_id,problem_id,date,category_id,level,solved,name,topics
0,40980,1001,15/02/25 06:15,1,1,127331,Extremely Basic,sequential
1,40980,1002,15/02/25 07:10,1,1,87459,Area of a Circle,sequential
2,40980,1003,15/02/25 06:17,1,1,89856,Simple Sum,sequential
3,40980,1004,15/02/25 06:19,1,1,85938,Simple Product,sequential
4,40980,1005,15/02/25 07:31,1,1,73427,Average 1,sequential


# Capítulo 3 - Tratamento dos dados

Inicialmente, para agilizar o processo de desenvolvimento, iremos trabalhar apenas com uma categoria.

In [2]:
filter_category = 7

categories = [
    'All', #0
    'Beginner', #1
    'Ad-Hoc', #2
    'String', #3
    'Libraries', #4
    'Math', #5
    'Paradigms', #6
    'Graph', #7
    'Geometry', #8
]

category = categories[filter_category]

if filter_category:
    solutions_df = solutions_df.loc[solutions_df['category_id'] == filter_category]

Quantidade de soluções do dataset:

In [3]:
len(solutions_df)

11403

Quantidade de problemas do dataset:

In [4]:
solutions_df['problem_id'].nunique()

181

Quantidade de usuários do dataset:

In [5]:
solutions_df['user_id'].nunique()

558

Quantidade de categorias do dataset:

In [6]:
solutions_df['category_id'].nunique()

1

# Capítulo 4 - Desenvolvimento

Uma série de técnicas de Machine Learning serão utilizadas a fim de obter o resultado final. Serão aqui descritas passo a passo, os procedimentos adotados.

obs: cada exemplo poderia conter o link de um problema

- storytelling

Em todo período acadêmico, o conteúdo aprendido é desenvolvido de forma gradual. Como as aulas de cálculo, onde temos o estudo de funções, limites, derivadas e integrais. Todo o conteúdo aprendido inicialmente é usado como base para os demais.

Da mesma forma funciona o treinamento para a maratona de programação. Primeiramente temos os problemas que envolvem estruturas de dados, como listas encadeadas. Posteriormente lidamos com árvores, grafos e etc.

Com isso podemos supor que todos os problemas resolvidos contribuem com o aprendizado do competidor em algum tópico. E ai está o nosso primeiro problema: **Identificar os tópicos necessários a se saber para a solução de um problema**

Tendo isso em mente, essa será a primeira tarefa desenvolvida. Para isso faremos o seguinte.

Seja p um problema e u um usuario do nosso dataset. Serão filtradas todas as soluções submetidas pelo usuário u antes da solução do problema p. Isso nos permitirá identificar quais os tópicos que foram desenvolvidos pelo usuário u que o capacitaram a solucionar o problema p.

Para isso iremos utilizar a técnica de Collaborative Filtering.

## Colaborative Filtering

Imagine um dataset composto por avaliações de filmes, onde cada usuário avalia um filme em uma nota entre 1 e 5. Teriamos uma tabela da seguinte forma:

| Filmes  | Ana | Beth | Carlos | Davi |
|---------|:---:|:----:|:------:|:----:|
| Querido John |  5  |   5  |    0   |   0  |
| Titanic |  5  |   5  |    0   |   0  |
| Batman |  0  |   0  |    5   |   5  |
| Thor |  0  |   0  |    5   |   5  |

Baseado na avaliação dos usuários podemos encontrar o gênero de cada filme.

| Filmes  | Ana | Beth | Carlos | Davi | Ação | Romance|
|---------|:--:|:--:|:--:|:--:|:--:|:--:|
| Querido John | 5 | 5 | 0 | 0 | ? | ? |
| Titanic | 5 | 5 | 0 | 0 | ? | ? |
| Batman | 0 | 0 | 5 | 5 | ? | ? |
| Thor | 0 | 0 | 5 | 5 | ? | ? |

Para que isso seja possível, existe um vetor de preferênca de cada usuário $\theta$ que indica o quanto cada usuário gosta de cada gênero de filme. E cada filme tem um vetor $X$ corresponde a quanto o filme pertence ao gênero indicado.

Podemos então estimar um valor de $\theta$ dos usuários a fim de descobrir o valor de $X$. Daí fazemos o contrário. É quase uma questão de quem nasceu primeiro, o ovo ou a galinha.

Descrever o algoritmo e o exemplo pra provar que funciona. :)

In [21]:
def get_solution_date(solutions_df, user_id, problem_id):
    solution = solutions_df[(solutions_df['problem_id'] == problem_id) & (solutions_df['user_id'] == user_id)]
    solution_date = solution['date'].values[0]

    return solution_date


def get_past_problems(solutions_df, user_id, problem_id):
    solution_date = get_solution_date(solutions_df, user_id, problem_id)
    past_solutions = solutions_df[(solutions_df['date'] < solution_date) & (solutions_df['user_id'] == user_id)]
    problems = past_solutions['problem_id']

    return list(problems)


def get_users_that_solved_the_problem(solutions_df, problem_id):
    solutions_of_the_problem_df = solutions_df[solutions_df['problem_id'] == problem_id]
    users = solutions_of_the_problem_df['user_id']
    
    return list(users)


def get_problems(solutions_df):
    problems = set(solutions_df['problem_id'])

    return list(problems)


def get_solution_matrix(solutions_df, problem):
    users = get_users_that_solved_the_problem(solutions_df, problem)

    solutions = []
    min_problem = 9999
    max_problem = 0

    for user in users:
        past_problems = get_past_problems(solutions_df, user, problem)

        solutions.append(past_problems)

        min_problem = min([min_problem] + past_problems)
        max_problem = max([max_problem] + past_problems)

    nr_problems = max_problem - min_problem + 1
    nr_users = len(users)
    matrix = np.zeros((nr_problems, nr_users))

    for user in range(nr_users):
        for problem in solutions[user]:
            problem_index = problem - min_problem
            matrix[problem_index][user] = 1 # ou level do problema
            
    return matrix

def optimize_matrix(matrix):
    matrix = matrix[~np.all(matrix == 0., axis=1)]
    
    return matrix


# http://www.bogotobogo.com/python/python_numpy_batch_gradient_descent_algorithm.php
def gradient_descent(x, y, alpha=0.01, nr_iterations=100):

    # m - número de amostras
    # n - número features
    m, n = x.shape

    # Insere coluna theta zero = 1
    x = np.c_[ np.ones(m), x]

    theta = np.ones(n + 1)
    x_transpose = x.transpose()

    for iter in range(0, nr_iterations):
        hypothesis = np.dot(x, theta)
        loss = hypothesis - y
        J = np.sum(loss ** 2) / (2 * m)
        gradient = np.dot(x_transpose, loss) / m         
        theta = theta - alpha * gradient

    return theta


NR_FEATURES = 4

def get_theta_of_problem(matrix):
    nr_problems, nr_users = matrix.shape

    theta_users = np.random.rand(nr_users, NR_FEATURES)
    theta_problems = np.zeros((nr_problems, NR_FEATURES))


    # matrix
    #     u1  u2  u3
    # p1   1
    # p2   1  1    1
    # p3      1

    matrix_t = np.transpose(matrix)
    
    print('nr_problems', nr_problems)
    print('nr_users', nr_users)
    
    i = 0
    while i < 1:
        i += 1

        for problem in range(nr_problems):
            x = np.array(
                theta_users
            )

            y = np.array(
                matrix[problem]
            )

            theta_problems[problem] = gradient_descent(x, y)[1:]

        for user in range(nr_users):
            x = np.array(
                theta_problems
            )

            y = np.array(
                matrix_t[user]
            )

            theta_users[user] = gradient_descent(x, y)[1:]
    
    return theta_problems.mean(axis=0)


def get_user_theta(problems_solved, thetas):
    

def get_next_problem(problems_solved, thetas):
    

In [20]:
thetas = {}

for problem in get_problems(solutions_df):
    print(problem)
    matrix = get_solution_matrix(solutions_df, problem)
    matrix = optimize_matrix(matrix)

    theta = get_theta_of_problem(matrix)
    
    thetas[problem] = theta

2048
nr_problems 151
nr_users 13
1
.......................................................................................................................................................,,,,,,,,,,,,,[[ 0.37210574  0.40084035  0.25162789  0.41297276]
 [ 0.56627221  0.49901304  0.44080392  0.58228977]
 [ 0.5211694   0.55729196  0.42947837  0.57432592]
 [ 0.54905327  0.56407885  0.46150243  0.59141301]
 [ 0.54905327  0.56407885  0.46150243  0.59141301]
 [ 0.54905327  0.56407885  0.46150243  0.59141301]
 [ 0.53887958  0.51947891  0.39753102  0.55299675]
 [ 0.56676345  0.52626581  0.42955508  0.57008385]
 [ 0.53887958  0.51947891  0.39753102  0.55299675]
 [ 0.56627221  0.49901304  0.44080392  0.58228977]
 [ 0.56627221  0.49901304  0.44080392  0.58228977]
 [ 0.53887958  0.51947891  0.39753102  0.55299675]
 [ 0.54905327  0.56407885  0.46150243  0.59141301]
 [ 0.52166064  0.58454473  0.41822952  0.56212   ]
 [ 0.47218771  0.48317694  0.38731409  0.48991736]
 [ 0.47218771  0.48317694  0.3873140

 [ 0.36382391  0.33107356  0.32172561  0.35269319]]
2056
nr_problems 160
nr_users 14
1
................................................................................................................................................................,,,,,,,,,,,,,,[[ 0.44040086  0.35416832  0.23284589  0.25256588]
 [ 0.58051509  0.47326751  0.43132969  0.37009093]
 [ 0.56806868  0.49877422  0.45922375  0.40707599]
 [ 0.57818365  0.50767294  0.45333863  0.42751621]
 [ 0.59571518  0.5008265   0.42269544  0.42504436]
 [ 0.58240786  0.53263266  0.46506448  0.44167601]
 [ 0.58240786  0.53263266  0.46506448  0.44167601]
 [ 0.57818365  0.50767294  0.45333863  0.42751621]
 [ 0.57901294  0.46629369  0.38723557  0.34673556]
 [ 0.59485428  0.50712595  0.43717041  0.40469095]
 [ 0.59485428  0.50712595  0.43717041  0.40469095]
 [ 0.58051509  0.47326751  0.43132969  0.37009093]
 [ 0.57818365  0.50767294  0.45333863  0.42751621]
 [ 0.58240786  0.53263266  0.46506448  0.44167601]
 [ 0.52921687  0.48249952

 [ 0.4570643   0.34036878  0.24533204  0.19600604]]
1550
nr_problems 154
nr_users 228
1
..........................................................................................................................................................,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,[[ 0.39196913  0.34837597  0.33674033  0.35558303]
 [ 0.36526419  0.34920865  0.32913283  0.33236251]
 [ 0.45438407  0.41921458  0.41856259  0.43581549]
 [ 0.47728212  0.47109338  0.46082635  0.43769067]
 [ 0.50399135  0.47643526  0.46649535  0.47614979]
 [ 0.3903929   0.35759461  0.34627883  0.35265093]
 [ 0.4602941   0.41609511  0.42276394  0.42422045]
 [ 0.40423549  0.36686644  0.34917735  0.36968325]
 [ 0.42136475  0.38056341  0.3681281   0.37104708]
 [ 0.45115327  0.40352068  0.40836449  0.41504247]
 [ 0.38787389  0.348

 [ 0.36449091  0.32697373  0.3151538   0.31733172]]
1552
nr_problems 156
nr_users 202
1
............................................................................................................................................................,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,[[ 0.33180111  0.33909056  0.32547814  0.32442962]
 [ 0.35639904  0.36018489  0.3422991   0.34080015]
 [ 0.35492511  0.34452693  0.33500093  0.34509166]
 [ 0.42799861  0.45608426  0.43668207  0.44273285]
 [ 0.45654782  0.47542103  0.46470529  0.47249627]
 [ 0.48342437  0.47929403  0.4961851   0.47908231]
 [ 0.35576806  0.37708557  0.36103841  0.35138226]
 [ 0.41250511  0.42794167  0.43175186  0.42958148]
 [ 0.36305388  0.37601169  0.36063996  0.37039074]
 [ 0.382822    0.39992866  0.37263275  0.36836517]
 [ 0.42107103  0.43540054  0.4282788   0.427

 [ 0.33405578  0.34157861  0.325714    0.32375492]]
2073
nr_problems 132
nr_users 7
1
....................................................................................................................................,,,,,,,[[ 0.26961213  0.20701726  0.33998579  0.54526215]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.35801613  0.29658604  0.43580655  0.65037828]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.35801613  0.29658604  0.43580655  0.65037828]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.44539587  0.41978077  0.43716979  0.68023328]
 [ 0.39971527  0.38385208  0.35640569  0.61150878]
 [ 0.39971

...

1502
nr_problems 62
nr_users 1
1
..............................................................,[[ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853  0.20113827]
 [ 0.46829984  0.34235298  0.89272853

2032
nr_problems 169
nr_users 38
1
.........................................................................................................................................................................,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,[[ 0.27442257  0.26783974  0.30862984  0.38662883]
 [ 0.37158768  0.35507676  0.44454333  0.49160034]
 [ 0.37159441  0.34285099  0.40983881  0.49700271]
 [ 0.44395514  0.4300364   0.48422322  0.51786052]
 [ 0.47935593  0.46654189  0.49110072  0.55068753]
 [ 0.47703433  0.47084472  0.49051361  0.55502735]
 [ 0.42242306  0.38831692  0.4231662   0.50653333]
 [ 0.40135526  0.41588588  0.45720191  0.52411024]
 [ 0.36798514  0.37787535  0.40949028  0.50566731]
 [ 0.43994784  0.40503731  0.47365516  0.51579643]
 [ 0.46545547  0.43428386  0.47543799  0.52163367]
 [ 0.3733433   0.36424943  0.42366681  0.49716642]
 [ 0.47450545  0.48642371  0.50730148  0.54221119]
 [ 0.47163327  0.48246071  0.48975676  0.54793336]
 [ 0.40923728  0.40986066  0.40389788  0.456

1525
nr_problems 151
nr_users 69
1
.......................................................................................................................................................,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,[[ 0.28479155  0.37119099  0.39698083  0.38776198]
 [ 0.36240433  0.40577851  0.4539526   0.42233982]
 [ 0.31578937  0.39948552  0.43548241  0.40565103]
 [ 0.40397354  0.48962373  0.53318329  0.50720106]
 [ 0.43578271  0.49750124  0.53001008  0.50531217]
 [ 0.45693184  0.53814752  0.57424088  0.54199143]
 [ 0.34417585  0.43069107  0.47718114  0.4685824 ]
 [ 0.43345862  0.5184292   0.5490682   0.52344877]
 [ 0.40130204  0.45730051  0.48665226  0.46413618]
 [ 0.39503412  0.46534956  0.48336101  0.47547979]
 [ 0.41553856  0.50782101  0.52661992  0.4971679 ]
 [ 0.38752158  0.43928622  0.48977505  0.47207284]
 [ 0.43372266  0.51490335  0.53554949  0.51461457]
 [ 0.42420817  0.5030606   0.53519318  0.50685146]
 [ 0.41702657  0.51776183  0.49

nr_problems 154
nr_users 67
1
..........................................................................................................................................................,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,[[ 0.35552528  0.33711251  0.46984397  0.3558168 ]
 [ 0.43475849  0.43367091  0.50968224  0.40688049]
 [ 0.41978293  0.37932111  0.51276083  0.42067938]
 [ 0.47126301  0.50398608  0.5458006   0.47432356]
 [ 0.52554213  0.51000005  0.55452567  0.51292256]
 [ 0.53605446  0.52959974  0.58378295  0.51212274]
 [ 0.44719223  0.47493405  0.52425092  0.48023542]
 [ 0.51403062  0.53885896  0.56605924  0.49142476]
 [ 0.46629918  0.45443541  0.52669498  0.44425216]
 [ 0.47773121  0.50371251  0.53412847  0.48695317]
 [ 0.49242493  0.51738458  0.55842549  0.49621659]
 [ 0.4758506   0.4597915   0.51523867  0.45490625]
 [ 0.52051992  0.53923354  0.57573761  0.49849772]
 [ 0.50572185  0.51673471  0.547738    0.5091925 ]
 [ 0.47140648  0.47092244  0.538005

.....................................................................................................................................................,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,[[ 0.32501011  0.3315455   0.27156236  0.3307279 ]
 [ 0.39052237  0.39551191  0.35513923  0.40081024]
 [ 0.3552938   0.36728427  0.3326573   0.3717239 ]
 [ 0.4538485   0.42490301  0.44625676  0.44400442]
 [ 0.47047509  0.45537385  0.44182929  0.45719977]
 [ 0.50789597  0.46989496  0.47340244  0.47210772]
 [ 0.38305508  0.39392354  0.36049571  0.39516717]
 [ 0.46866942  0.43968326  0.4398436   0.45155473]
 [ 0.40335013  0.39172219  0.37081073  0.40811441]
 [ 0.41608526  0.39245058  0.39415808  0.42651265]
 [ 0.46527629  0.43233209  0.43139875  0.45249775]
 [ 0.39598373  0.39649234  0.36175171  0.41289899]
 [ 0.49182707  0.46779532  0.47706904  0.49277533]
 [ 0.46627666  0.47465897  0.45358276  0.4850531 ]
 [ 0.45140355  0.40876508  0.41491944  0.4539313 ]
 [ 0.47705428  0.42603837  0.4333

In [27]:
for t in thetas:
    print(thetas[t].sum(), t)
#problems_solved = []

#next_problem = get_next_problem(problems_solved, thetas)

1.77952396224 2048
1.45877835923 1539
1.58331985186 2056
1.40175561755 1550
1.40277777346 1552
1.71889411748 2073
1.44638997523 1562
1.47382478045 1053
1.55290438128 2079
1.42957061879 1056
1.31738087668 1057
1.6558032716 2082
1.69947072902 2081
1.85139817326 2085
1.47851479591 2086
1.5153657351 2088
1.52586649388 1584
1.6419381068 2098
1.34976770916 1076
1.70532781902 2103
1.56139461719 1592
1.28002535213 1081
1.38210938902 1082
1.4187487077 1085
2.03083662695 2112
1.42664848163 2117
1.49463590812 2118
1.36177144516 1610
1.33712191475 1100
1.31737466537 2127
1.60697550636 2128
1.42632364846 1617
1.73325797785 2130
1.46030575668 2131
1.42857719251 1621
1.42362143793 1111
1.60059133139 1628
1.42195847683 1123
1.3125458857 1128
1.67236959802 2155
1.3575476659 1135
1.37435073088 1653
1.34041242647 1655
1.32561435862 1148
1.52395936524 2173
1.26759851254 1152
1.93877313816 1668
1.41503762285 1669
1.7657082358 2182
1.25501208262 1671
1.46894873916 2184
1.7431414894 1675
1.52467329245 1677
1